In [202]:
import pandas as pd
import geopandas
from shapely.geometry import Point, Polygon

## 1. Spatial join Boston Streetscore records with Boston Zoning data

### Read in Boston streetscore data with target column, convert lat and long to Point, create 'boston_gdf' table

In [203]:
boston_df = pd.read_csv("~/Desktop/ML1030/us_safety/boston_safety.csv")

In [204]:
boston_df.head(5)

city   latitude  longitude    q-score  safety
0  Boston  42.273857 -71.050980  36.497364       1
1  Boston  42.274235 -71.051178  33.081684       1
2  Boston  42.274429 -71.050896  21.050259       0
3  Boston  42.274529 -71.050491  12.665337       0
4  Boston  42.274685 -71.052315  21.554031       0

In [205]:
boston_gdf = pd.DataFrame(boston_df)
boston_gdf['Coordinates'] = list(zip(boston_gdf.longitude, boston_gdf.latitude))
boston_gdf['Coordinates'] = boston_gdf['Coordinates'].apply(Point)
boston_gdf = geopandas.GeoDataFrame(boston_gdf, geometry='Coordinates')

In [206]:
boston_gdf.head(5)

city   latitude  longitude    q-score  safety  \
0  Boston  42.273857 -71.050980  36.497364       1   
1  Boston  42.274235 -71.051178  33.081684       1   
2  Boston  42.274429 -71.050896  21.050259       0   
3  Boston  42.274529 -71.050491  12.665337       0   
4  Boston  42.274685 -71.052315  21.554031       0   

                            Coordinates  
0  POINT (-71.05098000000002 42.273857)  
1  POINT (-71.05117800000002 42.274235)  
2  POINT (-71.05089599999999 42.274429)  
3  POINT (-71.05049100000002 42.274529)  
4  POINT (-71.05231500000002 42.274685)

In [207]:
boston_gdf.shape

(229564, 6)

### Read in Boston Zoning data 'gfile_Boston' 
Data obtained from: http://bostonopendata-boston.opendata.arcgis.com/datasets/b601516d0af44d1c9c7695571a7dca80_0?geometry=-71.106%2C42.284%2C-70.975%2C42.307


In [208]:
gfile_Boston = geopandas.read_file("Zoning_Subdistricts.shp")
gfile_Boston.head(5)

OBJECTID          ZONE_                   DISTRICT MAPNO  \
0     54194             CC  Mission Hill Neighborhood    6D   
1     54195             WM  South Boston Neighborhood    4F   
2     54196            M-4               South Boston     4   
3     54197      D St. NDA  South Boston Neighborhood    4F   
4     54198  SUMMER ST. LI  South Boston Neighborhood    4F   

             ARTICLE  SUBDISTRIC                               Unique_Cod  \
0                 59    Business             Mission Hill Neighborhood CC   
1                 68  Industrial             South Boston Neighborhood WM   
2  Underlying Zoning  Industrial                         South Boston M-4   
3                 68   Mixed Use      South Boston Neighborhood D St. NDA   
4                 68  Industrial  South Boston Neighborhood Summer St. LI   

   FAR  Shape_STAr  Shape_STLe                      Zone_Desc  \
0  3.0           0           0           Community Commercial   
1  2.0           0           0       Waterfront Manufacturing   
2  4.0           0           0       Restricted Manufacturing   
3  2.0           0           0  Neighborhood Development Area   
4  3.0           0           0               Local Industrial   

                                            geometry  
0  POLYGON ((-71.09451646281816 42.33244043424968...  
1  POLYGON ((-71.03554411066089 42.33992389263747...  
2  POLYGON ((-71.0421385497493 42.3462569178347, ...  
3  POLYGON ((-71.04142638209957 42.34454863344008...  
4  POLYGON ((-71.03770255121084 42.33811666220058...

In [209]:
gfile_Boston.DISTRICT.unique()

array(['Mission Hill Neighborhood', 'South Boston Neighborhood',
       'South Boston', 'South End Neighborhood', 'Fenway Neighborhood',
       'Charlestown Neighborhood',
       'Harborpark: Dorchester Bay/Neponset River Waterfront',
       'Greater Mattapan Neighborhood', 'Jamaica Plain Neighborhood',
       'Roslindale Neighborhood', 'Dorchester Neighborhood',
       'Hyde Park Neighborhood', 'Allston/Brighton Neighborhood',
       'West Roxbury Neighborhood', 'Roxbury Neighborhood',
       'Government Center/Markets',
       'South Station Economic Development Area', 'Bulfinch Triangle',
       'North End Neighborhood', 'Boston Harbor',
       'Stuart Street District', 'Leather District',
       'Harborpark: North End Waterfront', 'Cambridge Street North',
       'Boston Proper', 'Harborpark: Charlestown Waterfront', 'Chinatown',
       'Central Artery Special', 'Audbon Circle Neighborhood',
       'Bay Village Neighborhood', 'Midtown Cultural',
       'North Station Economic Devel

In [210]:
gfile_Boston.DISTRICT.nunique()

37

In [211]:
gfile_Boston.ZONE_.nunique()

304

In [212]:
gfile_Boston.Unique_Cod.nunique()

540

### Converting CRS
Check and found that 'boston_gdf' table does not have a CRS, so we assign a CRS (WGS84) to 'boston_gdf' table.<br>
In this way, 'boston_gdf' and 'gfile_Boston' have the same CRS, i.e. WGS84, so they are ready to spatial join

In [213]:
print(boston_gdf.crs)

None


In [214]:
print(gfile_Boston.crs)

{'init': 'epsg:4326'}


In [215]:
boston_gdf.crs = {'init' :'epsg:4326'}

In [216]:
print(boston_gdf.crs)

{'init': 'epsg:4326'}


### Spatial join 'boston_gdf' and 'gfile_Boston', and create a new table 'boston' 

In [218]:
boston = geopandas.sjoin(boston_gdf, gfile_Boston, op="within")

In [219]:
boston.shape

(53401, 18)

In [220]:
boston.head(2)

city   latitude  longitude    q-score  safety  \
37  Boston  42.277477 -71.053329  21.189661       0   
40  Boston  42.277805 -71.053894  17.655205       0   

                                     Coordinates  index_right  OBJECTID  \
37  POINT (-71.05332900000001 42.27747700000001)           25     54219   
40                  POINT (-71.053894 42.277805)           25     54219   

                                           ZONE_  \
37  LOWER MILLS SHORELAND OPEN SPACE SUBDISTRICT   
40  LOWER MILLS SHORELAND OPEN SPACE SUBDISTRICT   

                                             DISTRICT         MAPNO ARTICLE  \
37  Harborpark: Dorchester Bay/Neponset River Wate...  4B-4D, 5F-5H     42A   
40  Harborpark: Dorchester Bay/Neponset River Wate...  4B-4D, 5F-5H     42A   

    SUBDISTRIC                                         Unique_Cod  FAR  \
37  Open Space  Harborpark: Dorchester Bay/Neponset River Wate...  NaN   
40  Open Space  Harborpark: Dorchester Bay/Neponset River Wate...  NaN   

    Shape_STAr  Shape_STLe             Zone_Desc  
37           0           0  Shoreland Open Space  
40           0           0  Shoreland Open Space

In [221]:
boston = boston[['city', 'latitude', 'longitude', 'q-score', 'safety',
       'Coordinates', 'OBJECTID', 'SUBDISTRIC']]
boston.head(2)

city   latitude  longitude    q-score  safety  \
37  Boston  42.277477 -71.053329  21.189661       0   
40  Boston  42.277805 -71.053894  17.655205       0   

                                     Coordinates  OBJECTID  SUBDISTRIC  
37  POINT (-71.05332900000001 42.27747700000001)     54219  Open Space  
40                  POINT (-71.053894 42.277805)     54219  Open Space

### Note that the 'boston' table has only 53401 rows.
Meaning only 23.26% of 'boston_gdf'(Boston Streetscore records) fall into the zones within 'gfile_Boston'

In [222]:
len(boston)/ len(boston_gdf) * 100

23.261922601104704

## 2. Stratified sampling based on both the portion of Toronto zoning class and the portion of target column - 'safety'
The 'safety' column was created in the 'Streetscore_create_target_label.ipynb' notebook
https://github.com/littlebeanbean7/Toronto_Streetscore/blob/master/code/Streetscore_create_target_label.ipynb

### Understanding Toronto zoning class portion
The data were provided by stakeholders

In [225]:
Toronto = pd.read_excel("~/Desktop/ML1030/Toronto_zoning_portion.xlsx")

In [226]:
Toronto 

zoneClass  portionSum
0            residential       49.89
1             open space       17.86
2  employment industrial       13.90
3             unassigned        5.83
4             commercial        5.34
5              utilities        5.08
6          institutional        2.10

###  We decided to take 20000 samples from 'boston' table. Therefore, the number of samples for each zoneClass should be as listed in below table's column 'outofTwentyKsample'

In [236]:
Toronto['outofTwentyKsample'] =  Toronto.portionSum / 100 * 20000
Toronto['outofTwentyKsample'] =  Toronto['outofTwentyKsample'].astype(int)
Toronto

zoneClass  portionSum  outofTwentyKsample
0            residential       49.89                9978
1             open space       17.86                3571
2  employment industrial       13.90                2780
3             unassigned        5.83                1166
4             commercial        5.34                1068
5              utilities        5.08                1016
6          institutional        2.10                 420

### Check 'subdistrict' types in 'gfile_Boston' table, and found that they do not exactly match with Toronto zone class type

In [237]:
gfile_Boston.SUBDISTRIC.unique()

array(['Business', 'Industrial', 'Mixed Use', 'Miscellaneous',
       'Open Space', 'Residential', 'Comm/Instit'], dtype=object)

### Matching strategy<br>
Toronto - Boston<br>
#### Sure:<br>
residential	- Residential<br>
open space - Open Space<br>
employment industrial - Industrial<br>
commercial - Business<br>
institutional - Comm/Instit<br>

#### Not Sure:<br>
unassigned - Miscellaneous and Mixed Use<br>
utilities - Miscellaneous and Mixed Use<br>

### Based on above matching strategy, the portion of each subdistrict we should sample from 'boston' is listed in 'boston_portion' table

In [241]:
Toronto['relativeBostonSubdistrict'] = ['Residential', 'Open Space', 'Industrial', 'Miscellaneous and Mixed Use', 'Business', 
                           'Miscellaneous and Mixed Use', 'Comm/Instit']
Toronto

zoneClass  portionSum  outofTwentyKsample  \
0            residential       49.89                9978   
1             open space       17.86                3571   
2  employment industrial       13.90                2780   
3             unassigned        5.83                1166   
4             commercial        5.34                1068   
5              utilities        5.08                1016   
6          institutional        2.10                 420   

     relativeBostonSubdistrict  
0                  Residential  
1                   Open Space  
2                   Industrial  
3  Miscellaneous and Mixed Use  
4                     Business  
5  Miscellaneous and Mixed Use  
6                  Comm/Instit

In [242]:
boston_portion = pd.DataFrame(Toronto.groupby('relativeBostonSubdistrict')['outofTwentyKsample'].sum().sort_values(ascending = False))

In [243]:
boston_portion

outofTwentyKsample
relativeBostonSubdistrict                      
Residential                                9978
Open Space                                 3571
Industrial                                 2780
Miscellaneous and Mixed Use                2182
Business                                   1068
Comm/Instit                                 420

### Understanding 'boston' subdistrict portion

In [244]:
boston.SUBDISTRIC.value_counts()

Residential      33749
Open Space        5273
Business          4650
Industrial        3452
Mixed Use         2973
Miscellaneous     1890
Comm/Instit       1414
Name: SUBDISTRIC, dtype: int64

In [245]:
boston.SUBDISTRIC.value_counts() / boston.SUBDISTRIC.count() * 100

Residential      63.199191
Open Space        9.874347
Business          8.707702
Industrial        6.464298
Mixed Use         5.567311
Miscellaneous     3.539260
Comm/Instit       2.647890
Name: SUBDISTRIC, dtype: float64

### Stratified sampling based on both the portion of Toronto zoning class and the portion of target column - 'safety'
Note that if we are to subsample 'boston' based on 'boston_portion', not all classe meet the quantity requirement:<br>
For SUBDISTRIC = Industrial, when safety = 1, there are only 505 samples, less than the ideal 1390 that we would like to sample.

#### Understanding 'boston' table's 'subdistrict' by 'safety'

In [247]:
boston.groupby(['SUBDISTRIC','safety']).count()

city  latitude  longitude  q-score  Coordinates  \
SUBDISTRIC    safety                                                     
Business      0        3374      3374       3374     3374         3374   
              1        1276      1276       1276     1276         1276   
Comm/Instit   0         875       875        875      875          875   
              1         539       539        539      539          539   
Industrial    0        2947      2947       2947     2947         2947   
              1         505       505        505      505          505   
Miscellaneous 0        1384      1384       1384     1384         1384   
              1         506       506        506      506          506   
Mixed Use     0        2236      2236       2236     2236         2236   
              1         737       737        737      737          737   
Open Space    0        3019      3019       3019     3019         3019   
              1        2254      2254       2254     2254         2254   
Residential   0       14326     14326      14326    14326        14326   
              1       19423     19423      19423    19423        19423   

                      OBJECTID  
SUBDISTRIC    safety            
Business      0           3374  
              1           1276  
Comm/Instit   0            875  
              1            539  
Industrial    0           2947  
              1            505  
Miscellaneous 0           1384  
              1            506  
Mixed Use     0           2236  
              1            737  
Open Space    0           3019  
              1           2254  
Residential   0          14326  
              1          19423

#### The number of samples we would like to take from 'boston' for each subdistrict's each 'safety' class are listed in below table's 'eachSafetyClass'

In [249]:
boston_portion['eachSafetyClass'] = boston_portion.outofTwentyKsample/2
boston_portion['eachSafetyClass'] = boston_portion['eachSafetyClass'].astype(int)
boston_portion

outofTwentyKsample  eachSafetyClass
relativeBostonSubdistrict                                       
Residential                                9978             4989
Open Space                                 3571             1785
Industrial                                 2780             1390
Miscellaneous and Mixed Use                2182             1091
Business                                   1068              534
Comm/Instit                                 420              210

#### Stratified sampling: Create a new table 'boston_safety_subsample'

In [251]:
boston_safety_subsample = pd.concat([
    boston[(boston['SUBDISTRIC'] == 'Residential') & (boston['safety']==1)].sample(n = 4989),
    boston[(boston['SUBDISTRIC'] == 'Residential') & (boston['safety']==0)].sample(n = 4989),
    boston[(boston['SUBDISTRIC'] == 'Open Space') & (boston['safety']==1)].sample(n = 1786),
    boston[(boston['SUBDISTRIC'] == 'Open Space') & (boston['safety']==0)].sample(n = 1786),
    boston[(boston['SUBDISTRIC'] == 'Industrial') & (boston['safety']==1)].sample(n=505),
    boston[(boston['SUBDISTRIC'] == 'Industrial') & (boston['safety']==0)].sample(n= 1390 + 1390 - 505),
    boston[(boston['SUBDISTRIC'].isin(['Miscellaneous', 'Mixed Use'])) & (boston['safety']==1)].sample(n=1091),
    boston[(boston['SUBDISTRIC'].isin(['Miscellaneous', 'Mixed Use'])) & (boston['safety']==0)].sample(n=1091),
    boston[(boston['SUBDISTRIC'] == 'Business') & (boston['safety']==1)].sample(n = 534),
    boston[(boston['SUBDISTRIC'] == 'Business') & (boston['safety']==0)].sample(n = 534),
    boston[(boston['SUBDISTRIC'] == 'Comm/Instit') & (boston['safety']==1)].sample(n = 210),
    boston[(boston['SUBDISTRIC'] == 'Comm/Instit') & (boston['safety']==0)].sample(n = 210)
])

#### Double check whether 'boston_safety_subsample' is really stratified sampling based both on column 'SUBDISTRIC' and column 'safety'

In [252]:
boston_safety_subsample.groupby(['SUBDISTRIC','safety']).count()

city  latitude  longitude  q-score  Coordinates  \
SUBDISTRIC    safety                                                    
Business      0        534       534        534      534          534   
              1        534       534        534      534          534   
Comm/Instit   0        210       210        210      210          210   
              1        210       210        210      210          210   
Industrial    0       2275      2275       2275     2275         2275   
              1        505       505        505      505          505   
Miscellaneous 0        428       428        428      428          428   
              1        439       439        439      439          439   
Mixed Use     0        663       663        663      663          663   
              1        652       652        652      652          652   
Open Space    0       1786      1786       1786     1786         1786   
              1       1786      1786       1786     1786         1786   
Residential   0       4989      4989       4989     4989         4989   
              1       4989      4989       4989     4989         4989   

                      OBJECTID  
SUBDISTRIC    safety            
Business      0            534  
              1            534  
Comm/Instit   0            210  
              1            210  
Industrial    0           2275  
              1            505  
Miscellaneous 0            428  
              1            439  
Mixed Use     0            663  
              1            652  
Open Space    0           1786  
              1           1786  
Residential   0           4989  
              1           4989

In [253]:
boston_safety_subsample.groupby(['SUBDISTRIC']).count()

city  latitude  longitude  q-score  safety  Coordinates  \
SUBDISTRIC                                                               
Business       1068      1068       1068     1068    1068         1068   
Comm/Instit     420       420        420      420     420          420   
Industrial     2780      2780       2780     2780    2780         2780   
Miscellaneous   867       867        867      867     867          867   
Mixed Use      1315      1315       1315     1315    1315         1315   
Open Space     3572      3572       3572     3572    3572         3572   
Residential    9978      9978       9978     9978    9978         9978   

               OBJECTID  
SUBDISTRIC               
Business           1068  
Comm/Instit         420  
Industrial         2780  
Miscellaneous       867  
Mixed Use          1315  
Open Space         3572  
Residential        9978

In [254]:
boston_safety_subsample.groupby(['safety']).count()

city  latitude  longitude  q-score  Coordinates  OBJECTID  SUBDISTRIC
safety                                                                        
0       10885     10885      10885    10885        10885     10885       10885
1        9115      9115       9115     9115         9115      9115        9115

In [255]:
boston_safety_subsample.head()

city   latitude  longitude    q-score  safety  \
25915   Boston  42.348724 -71.071495  28.634125       1   
1131    Boston  42.288597 -71.065681  26.525396       1   
121773  Boston  42.295563 -71.138023  29.614868       1   
7005    Boston  42.312973 -71.076004  30.076191       1   
6342    Boston  42.311348 -71.060051  26.038879       1   

                                 Coordinates  OBJECTID   SUBDISTRIC  
25915           POINT (-71.071495 42.348724)     54998  Residential  
1131    POINT (-71.06568100000001 42.288597)     55642  Residential  
121773          POINT (-71.138023 42.295563)     54450  Residential  
7005            POINT (-71.076004 42.312973)     55523  Residential  
6342            POINT (-71.060051 42.311348)     55604  Residential

#### Save the 'boston_safety_subsample' table. Next, would obtain Boston street view images using Google API based on this table.

In [256]:
boston_safety_subsample.to_csv("~/Desktop/ML1030/us_safety/boston_safety_subsample.csv")